In [320]:
import pandas as pd
import math

# Exercise 1

In [321]:
df = pd.read_csv('~/Downloads/ex1_table.csv')

In [322]:
df

,Airport Code,Lat,Long
0,CDG,49.012798,2.550000
1,CHC,-43.489399,172.531998
2,DYR,64.734901,177.740997
3,EWR,40.692501,-74.168701
4,HNL,21.318701,-157.921997
5,OME,64.512199,-165.445007
6,ONU,-20.650000,-178.699997
7,PEK,40.080101,116.584999


In [323]:
coords = {t[0]:(t[1],t[2]) for t in df.to_numpy()}

In [324]:
coords

{'CDG': (49.0127983093, 2.54999995232),
 'CHC': (-43.4893989562988, 172.53199768066398),
 'DYR': (64.7349014282227, 177.740997314453),
 'EWR': (40.6925010681152, -74.168701171875),
 'HNL': (21.3187007904053, -157.92199707031202),
 'OME': (64.5121994018555, -165.445007324219),
 'ONU': (-20.6499996185303, -178.699996948242),
 'PEK': (40.0801010131836, 116.584999084473)}

I would compute distances using the <b>[Haversine distance](https://en.wikipedia.org/wiki/Haversine_formula)</b> formula for latitude and longitude distance between two points on a sphere. 
* [Source 1](https://andrew.hedges.name/experiments/haversine/) 
* [Source 2](https://gist.github.com/rochacbruno/2883505)

In [325]:
def havdist(a,b):
    
    # function returns distance in kilometers
    
    lat1 = a[0]
    lat2 = b[0]
    
    lon1 = a[1]
    lon2 = b[1]
    
    ### average radius in km of Earth assuming it's a perfect sphere
    R = 6373 
    dlon = math.radians(lon2 - lon1)
    dlat = math.radians(lat2 - lat1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d = R*c
    
    return(d)

In [326]:
havdist(coords['CDG'],coords['CHC'])

19037.286815140535

In [327]:
def sorted_airports(coords,origin):
     
    candidates = [k for k in coords.keys() if k != origin]
     
    airports = {}
    for c in candidates:
        
        airports[c] = havdist(coords[origin],coords[c])
    
    nearest_airports = {a:airports[a] for a in sorted(airports, key=airports.get)}
    
    return(nearest_airports)
        

In [328]:
sorted_airports(coords,'PEK')

{'DYR': 4703.290435510359,
 'OME': 5502.753636192683,
 'HNL': 8134.981676799219,
 'CDG': 8191.316316210715,
 'ONU': 9508.285362066754,
 'CHC': 10856.149489614194,
 'EWR': 10971.319185764798}

# Exercise 2

This problem appears to be an A/B testing exercise, where we will be testing Bernoulli trials i.e. conversion rate using a pooled <b> [Z-test of proportions](https://stattrek.com/hypothesis-test/difference-in-proportions.aspx) </b>

In [329]:
import scipy.stats

df = pd.read_csv('~/Downloads/ex2_table.csv')

In [330]:
df

,device_type,variant,discount,total_views,price_freezes
0,android,Challenger,False,6010,189
1,android,Challenger,True,331,16
2,android,Champion,False,1084,23
3,android,Champion,True,54,3
4,iOS,Challenger,False,6905,336
5,iOS,Challenger,True,1986,196
6,iOS,Champion,False,6576,266
7,iOS,Champion,True,2054,161


In [331]:
#Calculating pooled proportion for hypothesis testing

p0 = df.sum().price_freezes/df.sum().total_views

In [332]:
df['price_freeze_rate'] = df.price_freezes/df.total_views

In [333]:
a = df.loc[df.variant == 'Champion']
b = df.loc[df.variant == 'Challenger']

combined = a.merge(b,how='inner',left_on=['device_type','discount'],right_on=['device_type','discount'],suffixes=('_a','_b'))

In [334]:
combined['diff'] = combined.price_freeze_rate_a - combined.price_freeze_rate_b

In [335]:
# creating a small function to compute the test statistic and one-sided p-value

def z_test(p0,p1,p2,n1,n2):
    z = (p2 - p1)/(math.sqrt(p0*(1-p0)*((1/n1) + 1/n2)))
    return(z)

def pvalue(z):
    p = scipy.stats.norm.sf(abs(z))
    return(p)

In [336]:
combined['z_stat'] =  combined.apply(lambda row: z_test(p0,row['price_freeze_rate_a'],row['price_freeze_rate_b'],row['total_views_a'],row['total_views_b']),axis=1)
combined['one_sided_p'] = combined.z_stat.apply(lambda z: pvalue(z))


In [337]:
# Looking at differences among all sub-populations to control for the confounding variables
combined

,device_type,variant_a,discount,total_views_a,price_freezes_a,price_freeze_rate_a,variant_b,total_views_b,price_freezes_b,price_freeze_rate_b,diff,z_stat,one_sided_p
0,android,Champion,False,1084,23,0.021218,Challenger,6010,189,0.031448,-0.010230,1.456007,0.072695
1,android,Champion,True,54,3,0.055556,Challenger,331,16,0.048338,0.007217,-0.230959,0.408673
2,iOS,Champion,False,6576,266,0.040450,Challenger,6905,336,0.048660,-0.008210,2.237927,0.012613
3,iOS,Champion,True,2054,161,0.078384,Challenger,1986,196,0.098691,-0.020307,3.030651,0.001220


In [338]:
overall = df.groupby(['variant']).sum()[['total_views','price_freezes']]
overall['rate'] = overall.price_freezes/overall.total_views

In [339]:
overall
### p0 is the same as before, as it's pooled probability

,total_views,price_freezes,rate
variant,,,
Challenger,15232,737,0.048385
Champion,9768,453,0.046376


In [340]:
### Looking at overall probabilities to answer the first question

overall_z = z_test(p0,0.046376,0.048385,9768,15232)
print('Overall z-score is {}'.format(overall_z))
print('Overall p-value is {}'.format(pvalue(overall_z)))

Overall z-score is 0.727910616681884
Overall p-value is 0.23333415188704776


In [341]:
### To answer the last question -- filtering on the Challenger model, where I'll use the pooled freeze rate to compute
### the confidence interval likelihood of observing a given number of price freezes

device = df.loc[(df.variant == 'Challenger') & (df.device_type == 'iOS')].groupby(['device_type']).sum()[['total_views','price_freezes']]
device['rate'] = device.price_freezes/device.total_views
device

,total_views,price_freezes,rate
device_type,,,
iOS,8891,532,0.059836


In [342]:
np = device['rate'][0]*10000
print('Expected price freezes using Challenger model for 10,000 views is {}'.format(round(np,0)))

Expected price freezes using Challenger model for 10,000 views is 598.0


In [343]:
# We assume we can use the Z-statistic because sample size is large enough

p_hat = 500/10000
p_expected = device['rate'][0] 

z = (p_hat - p_expected)/math.sqrt(p_expected*(1-p_expected)*(1/10000))

print('The Z test statistic is {}'.format(z))
print('P-value is {}'.format(pvalue(z)))


The Z test statistic is -4.146931246080383
P-value is 1.6848059696943557e-05


In [344]:
p_hat = 600/10000
p_expected = device['rate'][0] 

z = (p_hat - p_expected)/math.sqrt(p_expected*(1-p_expected)*(1/10000))

print('The Z test statistic is {}'.format(z))
print('P-value is {}'.format(pvalue(z)))


The Z test statistic is 0.06923407226160429
P-value is 0.4724016512116393


In [345]:
device['rate'][0]

0.05983578900011247

To answer the questions:

1. Based on the above, there is a <b>23%</b> chance we'd commit a Type I error and incorrectly reject the null hypothesis (that they're the same) in favor of the alternative hypothesis (that the Challenger model is better). Conversely, this means that we have a confidence of <b>77%</b> probability that the Challenger model is better compared to the Champion model.



2. Based on my answer to the first question, I am <i>not</i> comfortable rolling out the new model for every customer. <i>However</i>, I would like to note that <b> iOS users </b> across the board responded in a statistically significantly positive way to the new Challenger model at p-values of 1.2% (discount) and 0.1% (non-discount). Thus, I <b> would be comfortable rolling out the new Challenger model to iOS users </b> and then troubleshoot and try to identify the root cause behind the Android user population's relative under-response to the new model... perhaps it's related to the implementation on Android devices.



3. Given the current expected price freeze rate of <b>5.98%</b> for the iOS population that we've observed for these users and the large hypothetical sample size of 10,000 views, we can be very confident that we'll observe at least 500 price freezes for a sample size of 10,000 with the Challenger model (with probability close to 1). Given the same parameters, we can expect to observe  600 price freezes with a probability of <b> close to 47%</b>. This does, however, assume that the proportion of discount impressions among the test population is the same as the mix given here

# Exercise 3 

1. I'd transform this data using one-hot encoding of flight attributes. Specifically, I'd create a vectorization which contained the following attributes:

    i. <b>is_round_trip </b> dummy variable for whether a trip is one-way or round-trip
    
    ii. <b>departure_date</b> variable, transformed to an integer like UNIX Epoch time
        
    iii. <b>departure_{city/airport}_{CODE}</b> that had every city/airport code on the platform that was 1 if the user's search had the location as the origin, 0 if it didn't
    
    iv. <b>destination_{city/airport}_{CODE}</b> - same logic for departure, but for destination
    
    v. <b>stay_duration</b> - stay length as given in the file, only populated for round-trips it looks like -- we could pad the missing values for the one-way trips with a median, or we could do separate vectorizations for round-trip and one-way trips that do/don't have this attribute respectively (in which case we wouldn't need the "is_round_trip" dummy variable given in i. either
    
    vi. <b>return_date</b> same as departure date, but since it's only populated for round-trip searches it might make sense to split one-way/round-trip vectorizations 
    
    
I would bucket searches that satisfied the following criteria, averaging the vectors created above based on the criteria below to return a single "smoothed" search-trip vector:

1. <b> Same user_id </b>
2. <b> Same departure location (city/airport) </b>
3. <b>Departure date within a given range </b> -- say no greater than 3-7 days difference (the appropriate threshold would have to be determined from exploratory data analysis of the empirical proper threshold for distinct trip search buckets) -- from other searches they've conducted that meet the first 2 criteria


You can use methods such as <b>where</b>, <b>get_dummies</b>, <b> map</b>, or <b> apply</b> to accomplish the above. As I mentioned before, it might make sense to split the vectorizations into two separate processes for the one-way and round-trip searches since the number of available attributes may differ. On the other hand, we might get away with imputing an average/median value within each of the buckets above regardless of round-trip/one-way status -- users may, after all, choose to only search a segment of their entire trip on the platform (e.g. one-way SFO -> LAX on Hopper, booking LAX -> DFW on another platform, later returning to conduct a search for round-trip SFO -> DFW in its entirety) -- these trips could be bucketed and we could impute the round-trip values to the one-way trip values in these cases. This method isn't perfect, but I believe it'd get us closer to grouping searches and similar trips

2. <b> To compute overall similarity I'd use cosine similarity </b> which is analogous to the "correlation", which is the dot product of the search vectorizations divided by their respective norms. <b>[Source](https://en.wikipedia.org/wiki/Cosine_similarity)</b>


3. It would probably enhance trip similarity estimates if we included any intermediate layovers (not essential), arrival dates, departure/arrival times + hours, whether the purchased flights were for the user themselves or family/colleagues for business etc. It would also be great if we had carrier information, as folks fly with many different airlines, many of which leave/arrive on the same day from/to the same places


# Exercise 4

To measure the success of the "Price Drop" feature, I'd first want to discuss with the product team whether the proposed KPIs effectively captured the change they were hoping to implement:

* Daily/weekly bookings
* Daily/weekly/"Price drop" usage rates as a % of total weekly bookings
* Gross daily/weekly booking dollars net "Price Drop" refunds (also this metric using Hopper's commission net "Price Drop" refunds)

We'd want to ensure that we created a properly powered A/B test that rolled out the split test to a stratified, relatively balanced set of users (we don't want all of our power-users ending up in one test bucket) that ran for say... 2-4 weeks (based on Hopper's traffic, booking volume, etc. to ensure sufficient statistical power and the lag necessary for the "Price Drop" feature to be used)

The metrics above would help capture

   * whether overall bookings have increased relative to the control due to the introduction of the feature
   * whether the feature was being utilized
   * whether the feature was good for Hopper's business
   